# DOcplex ile Lineer Programlama Problemi Çözümü #

__Bu eğitimde basit bir problemi Docplex kullanarak çözmeye çalışacağız.__ <br>
__Eğer bilgisayarımızda docplex için gerekli kütüphaneler yok ise onları nasıl yükleyeceğimizi anlatacağız.__ <br>
__Önce problemimize bakalım sonrasında çözüm için adım adım ilerleyelim__ <br><br>


### Problem ###

Bir makarna fabrikası sadece 2 çeşit makarna üretmektedir. A ve B makarnaları. Her iki makarna çeşidinin üretimi için Süt ve Una ihtiyaç duyulmaktadır. Aşağıda A ve B makarnaları için gerekli olan miktarlar verilmiştir. <br>

    Her bir A makarnası için 1 ünite Süt ve 3 ünite Un
    Her bir B makarnası için 1 ünite Süt ve 2 ünite Un
    
Şirketimizin stoğunda 5 ünite Süt ve 12 ünite Un bulunmaktadır. Her bir satış sonrasında elde edilen kar aşağıda verilmiştir.

    Her bir A makarnasının satışında 7 TL
    Her bir B makarnasının satışından 5 TL
    
Firma karı maksimize etmek için, hangi makarnadan kaç adet üretmelidir?

__Problemimizde her şey açıkça belitildiği için, modelimizi daha kolay yazabilmek için, karar değişkenlerimizi, amaç fonksiyonumuzu ve kısıtlarımızı yazmak ile başlayalım__

#### Karar Değişkenleri ####
$ X $ --> A Makarnasından üretilecek miktar <br>
$ Y $ --> B Makarnasından üretilecek miktar

#### Amaç Fonksiyonu ####
___Maximize 7X + 5Y___

#### Kısıtlar ####
$ X + Y <= 5 $ <br>
$ 3X + 2Y <= 12 $ <br>
$ X >= 0 $ <br>
$ Y >= 0 $ <br>

### Problemin Docplex ile çözümü ###
Bu problemin çözümü için öncelikle gerekli kütüphanelerin ortamımızda kurulu olması gerekiyor. <br>
Anaconda Dağıtımı kuruluysa çoğu gerekli kütüphane hali hazırda gelecektir. İlk defa kurulum yapacaksanız bu adresten Anaconda Dağıtımını kurabilirsiniz. https://www.anaconda.com/products/distribution <br>
Docplex için ise 2 kütüphaneye ihtiyaç duyuyoruz: __cplex__ ve __docplex__ <br>


__cplex__ ve __docplex__ yüklemek için bir çok alternatifiniz bulunmaktadır. Herhangi bir prompt ekranı (cmd, anaconda, conda, vb.) açarak __pip install__ komutunu kullanarak bu kütüphaneleri yükleyebilirsiniz. 

```Python 
pip install cplex
pip install docplex
``` 

İlk olarak kütüphanelerimizi import ediyoruz. <br>
__cplex__ bilgisayarınızda yüklü olan cplex motorunu kullanmanızı sağlar. <br>
__docplex__ Cplex'i pythonda kullanmanızı sağlar. İki farklı kütüphane vardır. mp ve cp. <br>

__mp__ : Matematiksel Programlama - Mathematical Programming (docplex.mp)<br>
__cp__ : Kısıt Programlama - Constraint Programming (docplex.cp)

docplex.mp içindeki bazı classları daha rahat kullanmak için bunları ayrı import etmemiz faydalı olacaktır. Bunlardan en önemlisi __from docplex.mp.model import Model__ olandır. Optmizasyon modelimizi kurmamıza yarar. Karar değişkenlerimizi, kısıtlarımızı ve amaç fonksiyonumuzu ekleriz. Daha detaylı bilgi almak için http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html

### Kütüphanelerin alınması ###
Problemimiz basit olduğu için sadece bu 3 kütüphane ile çözüme ulaşabiliriz. Üst tarafta yazdığımız kütüphaneleri alıyoruz.

In [1]:
import cplex
import docplex.mp
from docplex.mp.model import Model

### Model parametreleri ###
Model class'ı 5 farklı parametre alabilmektedir. Biz aşağıda 2 tanesini kullanacağız.

__name (optional)__ – Modele vereceğiniz isimdir. <br>
__log_output (optional)__ – Eger bu parametre True olarak girilirse, model çalışırken ki log ekrana basılır. (Stdout) OPL'deki log mekanizmasının ekrana basılması olarak düşünebilirsiniz.  <br>

Biz modelimize __LP_Ornek__ adını verdik ve logları ekrana yazdımak istediğimiz için log_output parametresinin değerini True olarak girdik. 

In [62]:
model = Model(name='LP_Ornek', log_output=True)

### Karar Değişkenleri ###
Problemimizde 2 farklı karar değişkenimiz vardı. Bunlar sürekli değişkenler (continuous variable) oldukları için, modele continuous_var olarak ekliyoruz. 

__continuous_var__ 3 farklı parametre alabilmektedir. Kısaca bunları da inceleyelim.

__lb__ – Lower Bound yani alt sınır. Default değeri 0'dır. Karar değişkeninizin belli bir değerden büyük olmasını istiyorsanız, bunu kısıtlarda yazabileceğiniz gibi, burada da yazabilirsiniz. <br>
__ub__ – Upper Bound yani üst sınır. Default değeri Sonsuzdur. Karar değişkeninizin belli bir değerden küçük olmasını istiyorsanız, bunu kısıtlarda yazabileceğiniz gibi, burada da yazabilirsiniz.  <br>
__name (string)__ – Karar değişkene bir isim vermek istiyorsanız bu parametre ile yapabilirsiniz. <br>


In [63]:
x = model.continuous_var(name='x')
y = model.continuous_var(name='y')

### Amaç Fonksiyonu ###
Karar değişkenlerimizi tanımladıktan sonra sıra geldi, amaç fonksiyonunu yazmaya. Bu bir Lineer Programalama örneği olduğu için lineer bir ifade yazmamız gerekiyor. __maximize__ ya da __minimize__ yazdıktan sonra parantez açtıktan sonra lineer bir ifade yazarak çalıştırıyoruz.

In [64]:
model.maximize(7*x + 5*y)

### Kısıtlar ###
Modelimize kısıtlarımızı ekliyoruz. Kısıt eklemek için __add_constraint__ yazdıktan sonra parantez içine kısıtlarımızı giriyoruz. 

In [65]:
model.add_constraint(x + y <= 5)
model.add_constraint(3*x + 2*y <= 12); 

### Çalıştırmadan Önce Son Kontroller ###
Modelimizi oluşturduk, karar değişkenlerimizi belirledik, amaç fonksiyonumuzu ve kısıtlarımızı yadık. Modelimizi çözmeden önce mevcut durumu kontrol etmek için __print_information()__ çalıştırabiliriz. Sonuç olarak, 2 sürekli karar değişkenimiz, 2 kısıtımız var ve maximize yapmaya çalıştığımızı görüyoruz.

In [66]:
model.print_information() 

Model: LP_Ornek
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 2
   - linear=2
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


### Çözüm ###
Modelimizin çözümü için __solve()__ fonksiyonunu kullanıyoruz. Bunun sonucunda CPLEX motoru kullanılarak yazdığımız model çözülmektedir.

In [67]:
sol_model = model.solve()

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =            29.000000


### Sonuçların Yazılması ###
Model çözüm ürettikten sonra da bu sonuçlar __model__ içine kayıt edilir. __print_solution()__ kullanılarak modelin amaç fonksiyon sonucu ve karar değişkenlerinin hangi değerler aldığı görülebilir.

In [73]:
model.print_solution()

objective: 29.000
  x=2.000
  y=3.000


Model çalıştırırken sonuçları __sol_model__ parametresinin içine attığımız için bunu __print__ ederek de sonuçları görebiliri

In [75]:
print(sol_model)

solution for: LP_Ornek
objective: 29
x=2.000
y=3.000



Modelin içindeki karar değişkenlerini array içinde göstererek bakabilirsiniz.

In [77]:
sol_model[x]

2.0

In [78]:
sol_model[y]

3.0

Bu dersin sonuna geldik. Buraya kadar geldiğiniz için teşekkürler <br>
__Sabri Suyunu__